In [1]:
import tensorflow_hub as hub

In [2]:
import tensorflow as tf

In [3]:
detector = hub.load("https://tfhub.dev/google/yamnet/1")

In [4]:
tf.saved_model.save(detector, 'objmodel')

INFO:tensorflow:Assets written to: objmodel\assets


INFO:tensorflow:Assets written to: objmodel\assets


Available signatures:
__saved_model_init_op


TypeError: in user code:

    File "C:\Users\lewis\AppData\Local\Temp\ipykernel_45944\1551094031.py", line 17, in call  *
        scores, embeddings, spectrogram = self.yamnet_model(inputs)

    TypeError: Binding inputs to tf.function failed due to `Can not cast TensorSpec(shape=(None, None), dtype=tf.float32, name=None) to TensorSpec(shape=(None,), dtype=tf.float32, name=None)`. Received args: (<tf.Tensor 'inputs:0' shape=(None, None) dtype=float32>,) and kwargs: {} for signature: (waveform: TensorSpec(shape=(None,), dtype=tf.float32, name=None)).


In [20]:
import tensorflow as tf
import tensorflow_hub as hub

# Load the YAMNet model
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_layer = hub.KerasLayer(yamnet_model_handle, trainable=False)


In [23]:
class YAMNetWrapper(tf.keras.Model):
    def __init__(self, yamnet_layer):
        super(YAMNetWrapper, self).__init__()
        self.yamnet_layer = yamnet_layer

    @tf.function(input_signature=[tf.TensorSpec(shape=[None, None], dtype=tf.float32)])
    def call(self, inputs):
        # Directly pass batched inputs to YAMNet; ensure inputs are correctly shaped
        # YAMNet expects a batch of waveforms as [batch_size, waveform_length]
        scores, embeddings, spectrogram = self.yamnet_layer(inputs)
        # Process and return the desired output; for example, just return scores
        return scores


# Assuming yamnet_layer is your loaded YAMNet KerasLayer
yamnet_wrapper = YAMNetWrapper(yamnet_layer)


# Serving function that wraps the call to match expected serving signatures
@tf.function(input_signature=[tf.TensorSpec(shape=[None, None], dtype=tf.float32)])
def serving_fn(inputs):
    return {'scores': yamnet_wrapper(inputs)}


# Save the model with the defined serving function
tf.saved_model.save(yamnet_wrapper, 'saved_yamnet_model', signatures={'serving_default': serving_fn})


TypeError: in user code:

    File "C:\Users\lewis\AppData\Local\Temp\ipykernel_45944\1878317066.py", line 20, in serving_fn  *
        return {'scores': yamnet_wrapper(inputs)}
    File "C:\Users\lewis\PycharmProjects\yukaclone\.venv\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\lewis\AppData\Local\Temp\__autograph_generated_fileme9g8pl1.py", line 10, in tf__call
        (scores, embeddings, spectrogram) = ag__.converted_call(ag__.ld(self).yamnet_layer, (ag__.ld(inputs),), None, fscope)
    File "C:\Users\lewis\AppData\Local\Temp\__autograph_generated_filegerxav1s.py", line 74, in tf__call
        ag__.if_stmt(ag__.not_(ag__.ld(self)._has_training_argument), if_body_3, else_body_3, get_state_3, set_state_3, ('result', 'training'), 1)
    File "C:\Users\lewis\AppData\Local\Temp\__autograph_generated_filegerxav1s.py", line 37, in if_body_3
        result = ag__.converted_call(

In [22]:
# Instantiate the wrapper model
yamnet_wrapper = YAMNetWrapper(yamnet_layer)

# Save the model
tf.saved_model.save(yamnet_wrapper, 'saved_yamnet_model', signatures=yamnet_wrapper.call.get_concrete_function(
    tf.TensorSpec(shape=[None, None], dtype=tf.float32)))


OperatorNotAllowedInGraphError: in user code:

    File "C:\Users\lewis\AppData\Local\Temp\ipykernel_45944\544177672.py", line 10, in call  *
        batched_outputs = [self.yamnet_layer(tf.expand_dims(input, 0)) for input in inputs]

    OperatorNotAllowedInGraphError: Iterating over a symbolic `tf.Tensor` is not allowed. You can attempt the following resolutions to the problem: If you are running in Graph mode, use Eager execution mode or decorate this function with @tf.function. If you are using AutoGraph, you can try decorating this function with @tf.function. If that does not work, then you may be using an unsupported feature or your source code may not be visible to AutoGraph. See https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/limitations.md#access-to-source-code for more information.
